# A guide to start a simple CNN model in PyTorch

In [ ]:
import torch
from torch import nn
import numpy as np
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [6]:
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.Flatten=nn.Flatten()
        self.Dense=nn.Sequential(
            nn.Linear(28*28,256),
            nn.ReLU(),
            nn.Linear(256,10)
        )
        self.Softmax=nn.Softmax(dim=1)
    def forward(self, inputs):
        Flattened=self.Flatten(inputs)
        logits=self.Dense(Flattened)
        predictions=self.Softmax(logits)
        return predictions

In [17]:
def train_model(epochs, model, train_dataloader, loss_fn, optimizer):
    loss_epoch=[]
    for i in range(epochs):
        
        total_loss=0
        j=0
    
        for inputs, targets in train_dataloader:
            predictions=model(inputs)
            loss=loss_fn(predictions, targets)
            total_loss=total_loss+loss
            j=j+1
        loss_epoch.append(total_loss/j)
        print(f"Epoch={i+1} and Loss={total_loss/j}")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return model

In [30]:
def predict_model(model, test_dataloader, loss_fn,class_mapping):
    accuracy_t=[]
    for test_inputs, test_targets in test_dataloader:
        model.eval()
        with torch.no_grad():
            predictions=model(test_inputs)
            predicted_index=predictions.argmax(1)
            matching=torch.sum(predicted_index==test_targets)
            accuracy=matching/len(predicted_index)
            predicted=class_mapping[predicted_index[0]]
            expected=class_mapping[test_targets[0]]
            loss=loss_fn(predictions, test_targets)
            accuracy_t.append(accuracy)
    return(np.mean(accuracy_t))

In [29]:
class_mapping=[
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9"
]

In [22]:
def download_data():
    train_data=datasets.MNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor()
    )
    test_data=datasets.MNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor()
    )
    return train_data, train_data

train_data, test_data=download_data()

In [23]:
BATCH_SIZE=128

In [24]:
train_dataloader=DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader=DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [25]:
model=SimpleNN()

In [26]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=0.001)
epochs=5

In [27]:
model=train_model(epochs, model, train_dataloader, loss_fn, optimizer)

Epoch=1 and Loss=2.3021485805511475
Epoch=2 and Loss=2.294346332550049
Epoch=3 and Loss=2.287260055541992
Epoch=4 and Loss=2.2781646251678467
Epoch=5 and Loss=2.2671656608581543


In [31]:
predict_model(model, test_dataloader, loss_fn, class_mapping)

0.32379398